## 네이버 금융
- selenium
- iframe
    - HTML 안에 또 다른 HTML

In [41]:
import pandas as pd
import time
from selenium import webdriver

In [42]:
url = "https://finance.naver.com/marketindex/"
driver = webdriver.Chrome("../driver/chromedriver")
driver.get(url)

In [43]:
# 환전 고시 환율
contents = driver.find_elements_by_css_selector("#exchangeList > li > a") #element랑 elements 헷갈리지 않게 주의
contents[0].find_element_by_css_selector(".value").text

'1,188.50'

In [44]:
# 뉴스제목
news = driver.find_elements_by_css_selector("#content > div.section_news > div > ul > li:nth-child(1) > p > a")
news[0].text

'[오늘의 경제지표] 美 악재·원자재값 상승'

In [45]:
# 표 : 페이지 아래에 있는, 매매기준율
driver.find_elements_by_css_selector("body > div > table > tbody > tr") #tr 중 첫번째면 tr:nth-child(1) 인데 전부 가져올거니까 tr만
# id를 찾아보니 iframe으로 구분되어 있음

[]

# 1) iframe
- selenium을 이용해 접근하는 방법
    - iframe 태그 지정
    - ifrmae 태그 이동

In [46]:
# iframe 태그 지정
iframe = driver.find_element_by_css_selector("#frame_ex1") # driver.find_element_by_id("frame_ex1")


In [47]:
# iframe 태그 이동
driver.switch_to_frame(iframe)

C:\Users\82103\AppData\Local\Temp/ipykernel_4148/443683905.py:2: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame(iframe)


In [48]:
driver.find_element_by_css_selector(".sale").text

'1,188.50'

In [49]:
# 표 : 페이지 아래에 있는, 매매기준율
contents = driver.find_elements_by_css_selector("body > div > table > tbody > tr") #tr 중 첫번째면 tr:nth-child(1) 인데 전부 가져올거니까 tr만
# iframe을 이동해서 이번엔 가져와짐
contents[0].find_element_by_css_selector(".sale").text

'1,188.50'

In [50]:
## 간단하게 몇개만 가져와보자
datas = []
for content in contents[:10]:
    title = content.find_element_by_css_selector(".tit > a").text #통화명
    sale = content.find_element_by_css_selector(".sale").text
    link = content.find_element_by_css_selector(".tit > a").get_attribute("href") #href 속성 가져오기
    datas.append({
        "title" : title,
        "sale":sale,
        "link":link
    })

df = pd.DataFrame(datas)
df.to_excel("./sel_naver_finance.xlsx", encoding="utf-8")
df

,title,sale,link
0,미국 USD,"1,188.50",https://finance.naver.com/marketindex/exchange...
1,유럽연합 EUR,"1,377.59",https://finance.naver.com/marketindex/exchange...
2,일본 JPY (100엔),"1,068.94",https://finance.naver.com/marketindex/exchange...
3,중국 CNY,184.20,https://finance.naver.com/marketindex/exchange...
4,홍콩 HKD,152.67,https://finance.naver.com/marketindex/exchange...
5,대만 TWD,42.56,https://finance.naver.com/marketindex/exchange...
6,영국 GBP,"1,614.87",https://finance.naver.com/marketindex/exchange...
7,오만 OMR,"3,087.01",https://finance.naver.com/marketindex/exchange...
8,캐나다 CAD,941.50,https://finance.naver.com/marketindex/exchange...
9,스위스 CHF,"1,282.23",https://finance.naver.com/marketindex/exchange...


In [51]:
#driver.quit()

# 2) request 이용
- 접근 url을 알면 접근할 수 있다.

In [52]:
# 2) request 를 이용한 방법
import requests
from bs4 import BeautifulSoup

In [54]:
url = "https://finance.naver.com/marketindex/exchangeList.naver"
response = requests.get(url)
response

<Response [200]>

In [56]:
response.content
soup = BeautifulSoup(response.content, "html.parser")
print(soup.prettify())

<html lang="ko">
 <head>
  <title>
   네이버 금융
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="text/javascript" http-equiv="Content-Script-Type"/>
  <meta content="text/css" http-equiv="Content-Style-Type"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/css/finance.css" rel="stylesheet" type="text/css"/>
  <script language="javascript">
   document.domain="naver.com";
  </script>
  <script src="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript">
  </script>
  <script src="https://ssl.pstatic.net/imgstock/static.pc/20210916165954/js/lcslog.js" type="text/javascript">
  </script>
 </head>
 <body>
  <div class="tbl_area">
   <table border="1" class="tbl_exchange" summary="환전 고시 환율 리스트">
    <caption>
     환전 고시 환율
    </caption>
    <colgroup>
     <col width="162"/>
     <col width="92"/>
     <col width="92"/>
     <col width="92"/>
     <col widt

In [58]:
soup.select_one(".sale").text

'1,188.50'

In [59]:
contents = soup.select("tbody > tr") #부모 태그
contents[2].select_one(".sale").text

'1,068.84'

In [60]:
for content in contents:
    print(content.select_one(".sale").text) 

1,188.50
1,377.53
1,068.84
184.20
152.67
42.56
1,614.93
3,087.01
941.54
1,282.16
135.76
862.55
824.46
54.33
1.47
134.09
0.42
368.26
185.22
138.24
316.89
3,941.17
3,152.52
323.56
1,676.30
75.70
35.15
874.89
284.19
8.35
326.42
2.80
874.89
15.94
6.97
13.90
23.41
57.81
217.87
5.22
78.78
16.37
3.87
299.65
